In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, apply_category_mapping

C:\Users\user\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
 # Fetch adult dataset
adult = fetch_adult()
# Separate columns in numerical and categorical.
categorical_names = [adult.feature_names[i] for i in adult.category_map.keys()]
categorical_ids = list(adult.category_map.keys())
numerical_names = [name for i, name in enumerate(adult.feature_names) if i not in adult.category_map.keys()]
numerical_ids = [i for i in range(len(adult.feature_names)) if i not in adult.category_map.keys()]

# Split data into train and test
X, Y = adult.data, adult.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=13)
print(X_train[:5])

[[46  4  4  0  6  0  2  1  0  0 60  9]
 [28  4  4  1  2  4  4  0  0  0 40  5]
 [19  0  4  1  0  3  4  0  0  0 35  9]
 [43  4  4  1  2  3  4  1  0  0 40  9]
 [22  0  4  1  0  3  4  1  0  0 35  9]]


In [4]:
preprocessor = ColumnTransformer([("num", StandardScaler(), numerical_ids), 
                                  ("cat", OneHotEncoder(sparse=False, handle_unknown="ignore"),
                                   categorical_ids)])

In [5]:
preprocessor.fit(X_train)
# Preprocess train and test dataset.
X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [6]:
# Select one of the below classifiers.
# clf = XGBClassifier(min_child_weight=0.5, max_depth=3, gamma=0.2)
# clf = LogisticRegression(C=10)
# clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
clf = RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)
# Fit the classifier.
clf.fit(X_train_ohe, Y_train)

RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

In [11]:
# Define prediction function.
predictor = lambda x: clf.predict_proba(preprocessor.transform(x))

In [12]:
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.862


In [13]:
class ADULTEncoder(keras.Model):
    def __init__(self, hidden_dim: int, latent_dim: int, **kwargs):
        super().__init__(**kwargs)
        self.fc1 = keras.layers.Dense(hidden_dim)
        self.fc2 = keras.layers.Dense(latent_dim)
    
    def call(self, x: tf.Tensor, **kwargs) -> tf.Tensor:
        x = tf.nn.relu(self.fc1(x))
        x = tf.nn.tanh(self.fc2(x))
        return x

In [14]:
class ADULTDecoder(keras.Model):
    
    def __init__(self, hidden_dim: int, output_dims: List[int], **kwargs):
        
        super().__init__(**kwargs)

        self.fc1 = keras.layers.Dense(hidden_dim)
        self.fcs = [keras.layers.Dense(dim) for dim in output_dims]


    def call(self, x: tf.Tensor, **kwargs) -> List[tf.Tensor]:
        x = tf.nn.relu(self.fc1(x))
        xs = [fc(x) for fc in self.fcs]
        return xs


In [15]:
class HeAE(keras.Model):
    def __init__(self, encoder: keras.Model, decoder: keras.Model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def call(self, x: tf.Tensor, **kwargs):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [16]:
# Define attribute types, required for datatype conversion.
feature_types = {"Age": int, "Capital Gain": int, "Capital Loss": int, "Hours per week":int}

heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train, feature_names=adult.feature_names,
                                          category_map=adult.category_map,feature_types=feature_types)
# Define trainset
trainset_input = heae_preprocessor(X_train).astype(np.float32)
print(trainset_input.shape)
trainset_outputs = {"output_1": trainset_input[:, :len(numerical_ids)]}
for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({f"output_{i+2}": X_train[:, cat_id]})
trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(128, drop_remainder=True)

(26048, 57)


In [17]:
EPOCHS = 50 # epochs to train the autoencoder
HIDDEN_DIM = 128 # hidden dimension of the autoencoder
LATENT_DIM = 15 # define latent dimension
# Define output dimensions.
OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(adult.category_map[cat_id]) for cat_id in categorical_ids]
print(OUTPUT_DIMS)

[4, 9, 7, 4, 9, 6, 5, 2, 11]


In [19]:
# Define the heterogeneous auto-encoder.
heae = HeAE(encoder=ADULTEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
decoder=ADULTDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))
# Define loss functions.
he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]
# Add categorical losses.
for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))
# Define metrics.
metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})
# Compile model.
heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
loss=he_loss,
loss_weights=he_loss_weights,
metrics=metrics)

heae.fit(trainset, epochs=EPOCHS)

Epoch 1/50
203/203 [==============================] - 2s 2ms/step - loss: 1.1283 - output_1_loss: 0.2318 - output_2_loss: 1.1656 - output_3_loss: 1.1700 - output_4_loss: 0.6408 - output_5_loss: 1.5729 - output_6_loss: 0.9884 - output_7_loss: 0.5936 - output_8_loss: 0.3531 - output_9_loss: 0.6877 - output_2_sparse_categorical_accuracy: 0.6575 - output_3_sparse_categorical_accuracy: 0.6075 - output_4_sparse_categorical_accuracy: 0.7771 - output_5_sparse_categorical_accuracy: 0.4487 - output_6_sparse_categorical_accuracy: 0.6685 - output_7_sparse_categorical_accuracy: 0.8188 - output_8_sparse_categorical_accuracy: 0.8388 - output_9_sparse_categorical_accuracy: 0.8805
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 0.3951 - output_1_loss: 0.0422 - output_2_loss: 0.5100 - output_3_loss: 0.4868 - output_4_loss: 0.1598 - output_5_loss: 0.6723 - output_6_loss: 0.3369 - output_7_loss: 0.2322 - output_8_loss: 0.0936 - output_9_loss: 0.3317 - output_2_sparse_categorical_

203/203 [==============================] - 0s 2ms/step - loss: 0.0490 - output_1_loss: 0.0084 - output_2_loss: 0.0473 - output_3_loss: 0.0442 - output_4_loss: 0.0250 - output_5_loss: 0.0404 - output_6_loss: 0.0346 - output_7_loss: 0.0422 - output_8_loss: 0.0066 - output_9_loss: 0.0850 - output_2_sparse_categorical_accuracy: 0.9875 - output_3_sparse_categorical_accuracy: 0.9881 - output_4_sparse_categorical_accuracy: 0.9937 - output_5_sparse_categorical_accuracy: 0.9913 - output_6_sparse_categorical_accuracy: 0.9920 - output_7_sparse_categorical_accuracy: 0.9883 - output_8_sparse_categorical_accuracy: 0.9987 - output_9_sparse_categorical_accuracy: 0.9763
Epoch 14/50
203/203 [==============================] - 0s 2ms/step - loss: 0.0451 - output_1_loss: 0.0079 - output_2_loss: 0.0425 - output_3_loss: 0.0405 - output_4_loss: 0.0234 - output_5_loss: 0.0364 - output_6_loss: 0.0313 - output_7_loss: 0.0387 - output_8_loss: 0.0061 - output_9_loss: 0.0787 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 0s 2ms/step - loss: 0.0209 - output_1_loss: 0.0049 - output_2_loss: 0.0187 - output_3_loss: 0.0170 - output_4_loss: 0.0119 - output_5_loss: 0.0151 - output_6_loss: 0.0130 - output_7_loss: 0.0153 - output_8_loss: 0.0028 - output_9_loss: 0.0345 - output_2_sparse_categorical_accuracy: 0.9959 - output_3_sparse_categorical_accuracy: 0.9960 - output_4_sparse_categorical_accuracy: 0.9968 - output_5_sparse_categorical_accuracy: 0.9969 - output_6_sparse_categorical_accuracy: 0.9972 - output_7_sparse_categorical_accuracy: 0.9960 - output_8_sparse_categorical_accuracy: 0.9996 - output_9_sparse_categorical_accuracy: 0.9914
Epoch 26/50
203/203 [==============================] - 0s 2ms/step - loss: 0.0200 - output_1_loss: 0.0048 - output_2_loss: 0.0175 - output_3_loss: 0.0161 - output_4_loss: 0.0117 - output_5_loss: 0.0141 - output_6_loss: 0.0120 - output_7_loss: 0.0146 - output_8_loss: 0.0030 - output_9_loss: 0.0322 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 0s 2ms/step - loss: 0.0119 - output_1_loss: 0.0036 - output_2_loss: 0.0091 - output_3_loss: 0.0083 - output_4_loss: 0.0073 - output_5_loss: 0.0075 - output_6_loss: 0.0066 - output_7_loss: 0.0084 - output_8_loss: 0.0016 - output_9_loss: 0.0176 - output_2_sparse_categorical_accuracy: 0.9982 - output_3_sparse_categorical_accuracy: 0.9984 - output_4_sparse_categorical_accuracy: 0.9985 - output_5_sparse_categorical_accuracy: 0.9991 - output_6_sparse_categorical_accuracy: 0.9989 - output_7_sparse_categorical_accuracy: 0.9983 - output_8_sparse_categorical_accuracy: 0.9997 - output_9_sparse_categorical_accuracy: 0.9959
Epoch 38/50
203/203 [==============================] - 0s 2ms/step - loss: 0.0114 - output_1_loss: 0.0035 - output_2_loss: 0.0085 - output_3_loss: 0.0080 - output_4_loss: 0.0070 - output_5_loss: 0.0072 - output_6_loss: 0.0066 - output_7_loss: 0.0079 - output_8_loss: 0.0017 - output_9_loss: 0.0167 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 0s 2ms/step - loss: 0.0076 - output_1_loss: 0.0029 - output_2_loss: 0.0048 - output_3_loss: 0.0045 - output_4_loss: 0.0044 - output_5_loss: 0.0044 - output_6_loss: 0.0041 - output_7_loss: 0.0051 - output_8_loss: 0.0010 - output_9_loss: 0.0096 - output_2_sparse_categorical_accuracy: 0.9994 - output_3_sparse_categorical_accuracy: 0.9995 - output_4_sparse_categorical_accuracy: 0.9993 - output_5_sparse_categorical_accuracy: 0.9994 - output_6_sparse_categorical_accuracy: 0.9994 - output_7_sparse_categorical_accuracy: 0.9989 - output_8_sparse_categorical_accuracy: 0.9998 - output_9_sparse_categorical_accuracy: 0.9982
Epoch 50/50
203/203 [==============================] - 0s 2ms/step - loss: 0.0075 - output_1_loss: 0.0027 - output_2_loss: 0.0050 - output_3_loss: 0.0044 - output_4_loss: 0.0046 - output_5_loss: 0.0043 - output_6_loss: 0.0041 - output_7_loss: 0.0051 - output_8_loss: 0.0011 - output_9_loss: 0.0096 - output_2_sparse_categorical_accuracy: 

In [20]:
 # Define constants
COEFF_SPARSITY = 0.5 # sparisty coefficient
COEFF_CONSISTENCY = 0.5 # consisteny coefficient
TRAIN_STEPS = 10000 # number of training steps -> consider increasing the␣number of steps
BATCH_SIZE = 100 # batch size

Define dataset specific attributes and constraints
A desirable property of a method for generating counterfactuals is to allow feature conditioning. Real-world datasets
usually include immutable features such as Sex or Race, which should remain unchanged throughout the counterfactual
search procedure. Similarly, a numerical feature such as Age should only increase for a counterfactual to be actionable.

In [21]:
# Define immutable features.
immutable_features = ['Marital Status', 'Relationship', 'Race', 'Sex']
# Define ranges. This means that the `Age` feature can not decrease.
ranges = {'Age': [0.0, 1.0]}

In [22]:
explainer = CounterfactualRLTabular(predictor=predictor,
encoder=heae.encoder,
decoder=heae.decoder,
latent_dim=LATENT_DIM,
encoder_preprocessor=heae_preprocessor,
decoder_inv_preprocessor=heae_inv_preprocessor,
coeff_sparsity=COEFF_SPARSITY,
coeff_consistency=COEFF_CONSISTENCY,
category_map=adult.category_map,
feature_names=adult.feature_names,
ranges=ranges,
immutable_features=immutable_features,
train_steps=TRAIN_STEPS,
batch_size=BATCH_SIZE,
backend="tensorflow")

In [23]:
# Fit the explainer.
explainer = explainer.fit(X=X_train)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [06:29<00:00, 25.71it/s]


In [24]:
# Select some positive examples.
X_positive = X_test[np.argmax(predictor(X_test), axis=1) == 1]
X = X_positive[:1000]
Y_t = np.array([0])
C = [{"Age": [0, 20], "Workclass": ["State-gov", "?", "Local-gov"]}]

In [25]:
# Generate counterfactual instances.
explanation = explainer.explain(X, Y_t, C)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 48.95it/s]


In [26]:
# Concat labels to the original instances.
orig = np.concatenate(
[explanation.data['orig']['X'], explanation.data['orig']['class']],axis=1)
# Concat labels to the counterfactual instances.
cf = np.concatenate(
[explanation.data['cf']['X'], explanation.data['cf']['class']],axis=1)
# Define new feature names and category map by including the label.
feature_names = adult.feature_names + ["Label"]
category_map = deepcopy(adult.category_map)
category_map.update({feature_names.index("Label"): adult.target_names})
# Replace label encodings with strings.
orig_pd = pd.DataFrame(
apply_category_mapping(orig, category_map),
columns=feature_names)
cf_pd = pd.DataFrame(
apply_category_mapping(cf, category_map),
columns=feature_names)

In [27]:
orig_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K
1,35,Private,High School grad,Married,White-Collar,Husband,White,Male,7688,0,50,United-States,>50K
2,39,State-gov,Masters,Married,Professional,Wife,White,Female,5178,0,38,United-States,>50K
3,44,Self-emp-inc,High School grad,Married,Sales,Husband,White,Male,0,0,50,United-States,>50K
4,39,Private,Bachelors,Separated,White-Collar,Not-in-family,White,Female,13550,0,50,United-States,>50K
5,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,0,1902,40,?,>50K
6,50,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K
7,29,Private,Bachelors,Married,White-Collar,Wife,White,Female,0,0,50,United-States,>50K
8,47,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K
9,35,Private,Bachelors,Married,White-Collar,Husband,White,Male,0,0,70,United-States,>50K


In [28]:
cf_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,161,0,40,United-States,<=50K
1,35,Private,High School grad,Married,Blue-Collar,Husband,White,Male,149,0,50,United-States,<=50K
2,39,State-gov,Dropout,Married,Service,Wife,White,Female,488,0,37,United-States,<=50K
3,44,Self-emp-inc,High School grad,Married,Blue-Collar,Husband,White,Male,51,0,51,United-States,<=50K
4,39,Private,Bachelors,Separated,Admin,Not-in-family,White,Female,157,0,49,United-States,<=50K
5,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,0,1867,37,Latin-America,>50K
6,50,Private,Dropout,Married,Blue-Collar,Husband,White,Male,90,2,49,United-States,<=50K
7,29,Private,High School grad,Married,Blue-Collar,Wife,White,Female,347,1,50,United-States,<=50K
8,47,Private,Dropout,Married,Blue-Collar,Husband,White,Male,68,3,49,United-States,<=50K
9,35,Private,High School grad,Married,Blue-Collar,Husband,White,Male,163,0,70,United-States,<=50K


In [29]:
# Generate counterfactual instances.
X = X_positive[0].reshape(1, -1)
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=100,batch_size=10)

13it [00:00, 42.66it/s]


In [30]:
# Concat label column.
orig = np.concatenate(
[explanation.data['orig']['X'], explanation.data['orig']['class']],axis=1)
cf = np.concatenate(
[explanation.data['cf']['X'], explanation.data['cf']['class']],axis=1)
# Transfrom label encodings to string.
orig_pd = pd.DataFrame(
apply_category_mapping(orig, category_map),
columns=feature_names,
)
cf_pd = pd.DataFrame(
apply_category_mapping(cf, category_map),
columns=feature_names,
)

In [31]:
orig_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K


In [32]:
 cf_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,165,0,39,United-States,<=50K
1,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,169,0,39,United-States,<=50K
2,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,170,0,39,United-States,<=50K
3,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,170,0,40,United-States,<=50K
4,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,172,0,39,United-States,<=50K
